In [ ]:
import pandas as pd 
import pandas as pd
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import random

is_cuda_available = torch.cuda.is_available()
print(f"CUDA Available: {is_cuda_available}")

device = torch.device("cuda" if is_cuda_available else "cpu")
print(f"Using device: {device}")

directory='/home/slimhy/diffusion-timeseries/Diffusion/new_1m_binance'
all_highs = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)
        high_series = df['High']
        all_highs = pd.concat([all_highs, high_series.rename(filename)], axis=1)

all_highs.reset_index(drop=True, inplace=True)

all_highs

normalized_data = all_highs.apply(lambda x: -1 + 2 * (x - x.min()) / (x.max() - x.min()), axis=0)
normalized_data_tensor = torch.tensor(normalized_data.values).float().transpose(0, 1)
print(normalized_data_tensor.shape)

import matplotlib.pyplot as plt

normalized_data_cpu = normalized_data_tensor.cpu()

plt.figure(figsize=(10, 6))  
plt.plot(normalized_data_cpu[random.randint(0, 465)])  
plt.title('Real Time Series')  
plt.xlabel('Time')  
plt.ylabel('High Prices')  
plt.grid(True)  
plt.show()

input=normalized_data_tensor.unsqueeze(1)
print(input.size())

import matplotlib.pyplot as plt
import torch

timeseries = input[random.randint(0, 465), 0, :].numpy()

plt.figure(figsize=(10, 5))
plt.plot(timeseries)
plt.title('Timeseries Plot')
plt.xlabel('Timestep')
plt.ylabel('Value')
plt.grid(True)
plt.show()

import torch
from denoising_diffusion_pytorch import Unet1D, GaussianDiffusion1D, Trainer1D, Dataset1D

model = Unet1D(
    dim = 64, # You are free to tune this parameter
    dim_mults = (1, 2, 4, 8), # The number of layers can be reduced
    channels = 1
)

diffusion = GaussianDiffusion1D(
    model,
    seq_length = 1000, # 1000
    timesteps = 1000, # diffusion timesteps
    objective = 'pred_v'
)

dataset = Dataset1D(input)  # this is just an example, but you can formulate your own Dataset and pass it into the `Trainer1D` below

loss = diffusion(input)
loss.backward()

# Or using trainer

trainer = Trainer1D(
    diffusion,
    dataset = dataset,
    train_batch_size = 32,
    train_lr = 8e-5,
    train_num_steps = 100000,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True,                       # turn on mixed precision
)
trainer.train()

# after a lot of training

sampled_seq = diffusion.sample(batch_size = 100)
sampled_seq.shape

torch.save(model.state_dict(), 'model.pt')

series_index = random.randint(0, 99)
time_series = sampled_seq[series_index, 0, :] 
time_series = time_series.cpu().numpy()
time_axis = range(1000)

plt.plot(time_axis, time_series)
plt.xlabel("Time")
plt.ylabel("Value")
plt.title("Training Sequence Example")
plt.show()

torch.save(model.state_dict(), 'model.pt')
torch.save(diffusion.state_dict(), 'diffusion.pt')
